#Next Match

Find the next qualification match for 2158 that has not been played, and give some useful information



In [2]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

from common import canalytics

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Next_Match_Analysis')

tba_matches_by_team = pd.read_csv('./data/tba_matches_by_team.csv', index_col=0)
team_summary = pd.read_csv('./data/team_summary.csv', index_col=0)


Loading BokehJS ...

In [3]:
# Determine which match is next for 2158, and their alliances, and opponents
# TODO: Check to make sure this works.


In [17]:
next_match = min(tba_matches_by_team[ ( tba_matches_by_team['post_result_time'].isna() ) & (tba_matches_by_team['team']==2158)].match)
# ALl should be the same time.
predicted_time = tba_matches_by_team[ tba_matches_by_team['match'] == next_match ].predicted_time.values[0]
next_match

ValueError: min() arg is an empty sequence

In [6]:
# For each member:
alliances = tba_matches_by_team[ tba_matches_by_team['match'] == next_match ]
alliances = alliances.merge(team_summary,on="team",how='left')
alliances.sort_values(['alliance','team'],inplace=True)



In [14]:
# - OPRs
# - Average Cargo 
# For Alliance
# - Average CLimb points
# - previous OPR ( by flag )
# - previous District Info    
alliances=alliances.round(decimals=2)
alliances=alliances.fillna(0)

alliance_summary=alliances.groupby('alliance').agg(
                    avg_climb_points = pd.NamedAgg(column='climb_points_avg',aggfunc='sum'),
                    avg_cargo = pd.NamedAgg(column='total_cargo_avg',aggfunc='sum')
)


source = ColumnDataSource(alliances)


columns = [
        TableColumn(field="alliance", title="Alliance"),
        TableColumn(field="team", title="Team"),
        TableColumn(field="oprs", title="OPRS (Event)"),
        #TableColumn(field="pre_oprs", title="OPRS (Prior)"),
        TableColumn(field="total_cargo_avg", title="Total Cargo Avg"),
        TableColumn(field="auto_cargo_avg", title="Auto Cargo Avg"),
        TableColumn(field="teleop_cargo_avg", title="Teleop Cargo Avg"),
        TableColumn(field="highest_endgame_position", title="Highest",width=155),
        TableColumn(field="common_endgame_position", title="Common",width=155),
        TableColumn(field="climb_points_avg", title="Climb Pts Avg"),
        #TableColumn(field="pre_climb_points", title="Prior Climb Pts")
      ]

next_match_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True)

print(f"Next match: {next_match} at predicted time: {predicted_time}\n")


Next match: 53 at predicted time: 2022-04-02 10:21



In [15]:
alliance_summary

,avg_climb_points,avg_cargo
alliance,,
blue,11.67,10.17
red,13.00,10.64


In [16]:
show(next_match_table)